<a href="http://landlab.github.io"><img style="float: left" src="https://raw.githubusercontent.com/landlab/tutorials/master/landlab_header.png"></a>

# Using Landlab's gradient and flux divergence functions

Models of geophysical flow phenomena very commonly include *gradient* terms and *flux divergence* terms in their governing equations. Consider the example of conductive heat diffusion. The *flux* of heat at a point depends on the *gradient* in temperature:

$q = -k dT/dx$

$dT/dt \propto -dq/dx$

In a finite-difference or finite-volume solution, the quantities $T$ and $q$ are solved at discrete points. Staggered-grid schemes will place the scalar quantity ($T$) at one set of grid locations, and the vector quantity ($q$) at "in-between" locations. In Landlab, such a staggered grid is easy to implement by locating scalar quantities at either nodes or cells, and locating vector quantities at either links or faces. Landlab also provides a set of "one-liner" functions to calculate the gradients in a quantity located at nodes or cells, as well as to calculate the net flux or flux divergence in a vector quantity located at faces or links. This tutorial shows you examples of how to use these.

We'll start with a tiny grid, so it's easy to see what the various flavors of the gradient and flux functions do. Then we'll try them out on the example of a fault scarp undergoing soil creep, which is the same example used in an introductory Landlab tutorial.

## Tiny grid example

We'll start with a 3-row by 4-column raster grid, with 10-meter node spacing.

In [ ]:
import numpy
from landlab import RasterModelGrid
mg = RasterModelGrid((3, 4), 10.0)

Create a scalar field called `z` at the grid nodes:

In [ ]:
z = mg.add_zeros('node', 'topographic__elevation')
z[5] = 5.0
z[6] = 3.6

Nodes 5 and 6 are the only core nodes; the rest are (so far) open boundaries. Here's what the grid of `z` values looks like so far:

Now we'll add a *data field* to the grid, to represent the elevation values at grid nodes. The "dot" syntax indicates that we are calling a function (or *method*) that belongs to the *RasterModelGrid* class, and will act on data contained in **mg**. The arguments indicate that we want the data elements attached to grid nodes (rather than links, for example), and that we want to name this data field `land_surface__elevation`. The `add_zeros` method returns the newly created Numpy array.

In [ ]:
z = mg.add_zeros('node', 'land_surface__elevation')

Let's take a look at the grid we've created. To do so, we'll use the Pylab graphics library (imported under the name `plt`). We also have to tell the iPython Notebook to display plots right here on the page.

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt

Let's plot the positions of all the grid nodes. The nodes' *(x,y)* positions are stored in the arrays `mg.node_x` and `mg.node_y`, respectively.

In [ ]:
plt.plot(mg.node_x, mg.node_y, '.')

If we bothered to count, we'd see that there are 1000 grid nodes (25 x 40). The `z` array also has 1000 entries: one per grid cell.

In [ ]:
len(z)

Now for some tectonics. Let's say there's a fault trace that angles roughly east-northeast. We can describe the trace with the equation for a line. One trick here: by using `mg.node_x`, we are calculating a *y* (i.e., north-south) position of the fault trace for each grid node---meaning that this is the *y* coordinate of the trace at the *x* coordinate of a given node.

In [ ]:
fault_trace_y = 50.0 + 0.25*mg.node_x

We find the ID numbers of the nodes north of the fault trace with help from Numpy's `where()` function.

In [ ]:
upthrown_nodes = numpy.where(mg.node_y > fault_trace_y)

Here comes the earthquake. For all the nodes that we identified as being north of the fault, we'll add elevation equal to 10 meters plus a centimeter for every meter east along the grid (just to make it interesting).

In [ ]:
z[upthrown_nodes] += 10.0 + 0.01*mg.node_x[upthrown_nodes]

Let's look at our newly created initial topography using Landlab's *imshow_node_grid* plotting function (which we first need to import).

In [ ]:
from landlab.plot.imshow import imshow_node_grid

In [ ]:
imshow_node_grid(mg, 'land_surface__elevation')

To finish getting set up, we will define two parameters: the transport ("diffusivity") coefficient, `D`, and the time-step size, `dt`. (The latter is set using the Courant condition for a forward-time, centered-space finite-difference solution.)

In [ ]:
D = 0.01  # m2/yr transport coefficient

In [ ]:
dt = 0.2 * mg.dx * mg.dx / D

In [ ]:
dt

Boundary conditions: for this example, we'll assume that the east and west sides are closed to flow of sediment, but that the north and south sides are open. (Note: the order is east, north, west, and south)

In [ ]:
mg.set_closed_boundaries_at_grid_edges(False, True, False, True)

In a moment we will calculate changes in elevation for the inner or **core** nodes---those that are not boundaries---and so we'll need to remember their ID numbers.

In [ ]:
core_nodes = mg.core_nodes

In [ ]:
len(core_nodes)

And now for some landform evolution. We will loop through 25 iterations, representing 50,000 years. On each pass through the loop, we do the following:

1. Calculate, and store in the array `g`, the gradient between each neighboring pair of nodes. These calculations are done on **active links**.

2. Calculate, and store in the array `qs`, the sediment flux between each adjacent pair of nodes by multiplying their gradient by the transport coefficient.

3. Calculate, and store in `dqsdx`, the resulting net flux at each node (positive=net outflux, negative=net influx).

4. The rate of change of node elevation, `dzdt`, is simply `-dqsdx`.

5. Update the elevations for the new time step.

In [ ]:
for i in range(25):
    g = mg.calculate_gradients_at_active_links(z)
    #TO BE CHANGED TO: 
    #gf = mg.calculate_gradients_at_faces(z)
    qs = -D*g
    dqsdx = mg.calculate_flux_divergence_at_nodes(qs)
    #TO BE CHANGED TO: 
    #dqsda = mg.calc_face_flux_divergence_at_node(-D * gf)
    dzdt = -dqsdx
    #TO BE CHANGED TO: dzdt = -dqsda
    z[core_nodes] += dzdt[core_nodes]*dt

Let's look at how our fault scarp has evolved.

In [ ]:
imshow_node_grid(mg, 'land_surface__elevation')

Notice that we have just created and run a 2D model of fault-scarp creation and diffusion with fewer than two dozen lines of code. How long would this have taken to write in C or Fortran?

Complete code can be found here: https://github.com/landlab/tutorials/blob/master/fault_scarp/landlab-fault-scarp.py

### Click here for more <a href="https://github.com/landlab/landlab/wiki/Tutorials">Landlab tutorials</a>